In [ ]:
import pandas as pd;import numpy as np;import sys; 
from datetime import datetime
from datetime import timedelta

#csv サンプル synop_test.csv
#PrecTermは過去◯時間の合計降水量の◯時間部分
#precは過去◯時間の合計降水量
#utc,PrecTerm,prec
#2023-08-01 06:00:00,6,1
#2023-08-01 12:00:00,12,3
#2023-08-01 18:00:00,18,10


df_org = pd.read_csv("/usr/amoeba/pub/synop/data/test/synop_test.csv")

df_org['utc'] = pd.to_datetime(df_org['utc'])

utc_list = sorted(df_org["utc"].value_counts().index)

df_all = pd.DataFrame()

for (i,utc) in enumerate(utc_list,1):

   if i == 1:

      df1 = df_org[df_org["utc"]==utc]

      hour_interval = df1["PrecTerm"].iloc[0]

      total_prec = df1["prec"].iloc[0]

      prec_1hour = total_prec/hour_interval

      first_data = [] 
      
      for i in range(0,hour_interval):
         
         valid_utc = utc + timedelta(hours=-i)
         
         first_data.append([valid_utc,prec_1hour])
         
      df_first = pd.DataFrame(first_data).rename(columns={0:'utc',1:'prec'})

      df_all = df_first
      
   else:

      df2 = df_org[df_org["utc"]==utc]

      hour_interval2 = df2["PrecTerm"].iloc[0]

      start_utc2 = utc - timedelta(hours = int(hour_interval2))
      
      include_first = df_all[(df_all["utc"]>=start_utc2)&(df_all["utc"]<=utc)]
      
      
      include_total_prec = include_first["prec"].sum()

      total_prec2 = df2["prec"].iloc[0]

      first_end_time = include_first["utc"].max()

      time_interval2 = int((utc - first_end_time)/np.timedelta64(1,'h'))
      
      value_timestep2 = (total_prec2 - include_total_prec)/time_interval2

      second_data = []

      for i in range(0,time_interval2):
         
         ft_utc2 = utc + timedelta(hours=-i)
         
         second_data.append([ft_utc2, value_timestep2])
         
      df_second_data = pd.DataFrame(second_data).rename(columns={0:'utc',1:'prec'})
      
      df_all = pd.concat([df_all, df_second_data])
      
df_all.sort_values(by=["utc"], ascending=True)